In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install 'kaggle-environments==0.1.6'

In [ ]:
from collections import defaultdict
import numpy as np

In [ ]:
from kaggle_environments import evaluate, make, utils

env = make("connectx", debug=True)
#env.render()


In [ ]:
state_key = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                      0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]).astype(str) 
print(int(''.join(state_key)) )#空いてる所消す
print(int(''.join(state_key), 3) ) #3進数として解釈して10進に直している
print(hex(int(''.join(state_key), 3)))
s = hex(int(''.join(state_key), 3))[2:]  #0xは切り離す
print(state_key, s)

In [ ]:
def my_agent(observation, configuration):
    from random import choice
    return choice([c for c in range(configuration.columns) if observation.board[c] == 0])

In [ ]:
env = make("connectx", debug=True)
trainer = env.train([None, "random"])

class RandomAgent:
    def __init__(self):
        #self.env = env envはいったん外付けで
        self.gamma = 1
        self.action_size = 7

        random_actions = {0:1/7, 1:1/7, 2:1/7, 3:1/7, 4:1/7, 5:1/7, 6:1/7}

        self.pi = defaultdict(lambda: random_actions)
        self.v = defaultdict(lambda: 0)
        self.cnts = defaultdict(lambda: 0)
        self.memory = [] # state_key, action, reward
        
    def generate_state_key(self, observation):
        state = observation.board[:]
        state.append(observation.mark)
        state_key = np.array(state).astype(str) 

        return hex(int(''.join(state_key), 3))[2:]

    def get_action(self, observation):
        #print("enter")
        state = observation.board #埋まっている部分の可能性は他のところに山分けする
        
        

        state_key = self.generate_state_key(observation)
        #おけない部分の修正
        #山分けは誤差でばグルのでやめる
        p = 0
        tmp = {0:-1, 1:-1, 2:-1, 3:-1, 4:-1, 5:-1, 6:-1}
        for i in range(self.action_size):
            #print("state", state[i])
            if state[i] != 0:
                #print("bpi", self.pi[1])
                tmp[i] = 0
                #print("pi", self.pi[1])
            else:
                
                p += 1
       # print("middle", p)
        
        for i in range(self.action_size):
            if state[i] == 0:
                tmp[i] = 1 / p
                
        self.pi[state_key] = tmp
        action_probs = self.pi[state_key]
        
        
        actions = list(action_probs.keys())
        probs = list(action_probs.values())
        
        
        #if p != self.action_size and sum(probs) != 1:
            #print("before")
            #print(probs, sum(probs), state, p, self.pi[1])
            #print("after")
        
        #print("b",sum(probs),probs)
        return np.random.choice(actions, p=probs).tolist()

    def add(self, observation, action, reward):
        state_key = self.generate_state_key(observation)
        data = (state_key, action, reward)
        self.memory.append(data)

    def reset(self):
        self.memory.clear()

    def eval(self):
        G = 0
        for data in reversed(self.memory):
            state_key, action, reward = data
            G = self.gamma * G + reward
            self.cnts[state_key] += 1
            self.v[state_key] += (G - self.v[state_key]) / self.cnts[state_key]


In [53]:
env = make("connectx", debug=True)
trainer = env.train([None, "random"])
#　遷移確率は無い（完全にa,sに依存）し、
# 報酬もゲームが終了するか否かの判定によって得られるためQ関数を用いる必然性はないが…
class McAgent:
    def __init__(self):
        self.gamma = 1
        self.action_size = 7
        self.epsilon = 0.1
        self.alpha = 0.9
        random_actions = {action: 1/7 for action in range(self.action_size)}

        self.pi = defaultdict(lambda: random_actions)
        self.Q = defaultdict(lambda: 0)
        self.cnts = defaultdict(lambda: 0)
        self.memory = []
    
    def generate_state_key(self, observation):
        state = observation.board[:]
        state.append(observation.mark)
        state_key = np.array(state).astype(str) 

        return hex(int(''.join(state_key), 3))[2:]

    
    def get_action(self, observation):
        #行動はpiから取得
        state = observation.board #埋まっている部分の可能性は他のところに山分けする

        state_key = self.generate_state_key(observation)
        #おけない部分の修正
        #山分けをあきらめない
        p = 0
        pool = 0
        tmp = {0:-1, 1:-1, 2:-1, 3:-1, 4:-1, 5:-1, 6:-1}
        for i in range(self.action_size):
           
            if state[i] != 0:
                #print("captured")
                tmp[i] = 0 
                pool += self.pi[state_key][i]
            else:
                p += 1
        
        pool = pool / p
       
            
        for i in range(self.action_size):
            if state[i] == 0:
                tmp[i] = pool + self.pi[state_key][i]
            else:
                tmp[i] = 0
        
        #if p != self.action_size:
            #print("challenge")
            #print("before", self.pi[state_key])
            #print("after", tmp)
       
                
        self.pi[state_key] = tmp
        action_probs = self.pi[state_key]
        #print(action_probs)
       
        
        actions = list(action_probs.keys())
        probs = list(action_probs.values())
        
        return np.random.choice(actions, p=probs).tolist()
    
    def add(self, observation, action, reward):
        state_key = self.generate_state_key(observation)
        data = (state_key, action, reward)
        #print(data)
        self.memory.append(data)

    def reset(self):
        self.memory.clear()
    
    def greedy_probs(self, Q, state_key):
        #print("greedy", state_key)
        p = 0
        tmp = []
        for action in range(self.action_size):
            if self.pi[state_key][action] == 0:
                tmp.append(0)
            else:
                tmp.append(1)
                p += 1
        qs = [Q[(state_key, action)] for action in range(self.action_size)]
        max_action = np.argmax(qs)
        #おけない場所
        base_prob = self.epsilon / p
        #だめそうだったらprob作ってから
        action_probs = {action: 0.0 for action in range(self.action_size)}
        for action in range(self.action_size):
            if tmp[action] == 1:
                action_probs[action] += base_prob
        
        action_probs[max_action] += 1 - self.epsilon
        #print("before greedy", self.pi[state_key])
        #print("afrer greedy", action_probs)
        
        return action_probs
    
    def update(self):
        G = 0
        for data in reversed(self.memory):
            state_key, action, reward = data
            G = self.gamma * G + reward
            key = (state_key, action)
            self.cnts[key] += 1 
            self.Q[key] += (G - self.Q[key]) * self.alpha
            self.pi[state_key] = self.greedy_probs(self.Q, state_key)
    
    

In [77]:
env = make("connectx", debug=True)
trainer = env.train([None, "random"])
#　遷移確率は無い（完全にa,sに依存）し、
# 報酬もゲームが終了するか否かの判定によって得られるためQ関数を用いる必然性はないが…
class McOffPolicyAgent:
    def __init__(self):
        self.gamma = 1
        self.action_size = 7
        self.epsilon = 0.1
        self.alpha = 0.9
        random_actions = {action: 1/7 for action in range(self.action_size)}
        self.pi = defaultdict(lambda: random_actions)
        self.b = defaultdict(lambda: random_actions)
        self.Q = defaultdict(lambda: 0)
        self.cnts = defaultdict(lambda: 0)
        self.memory = []
        
    def generate_state_key(self, observation):
        state = observation.board[:]
        state.append(observation.mark)
        state_key = np.array(state).astype(str) 

        return hex(int(''.join(state_key), 3))[2:]

    
    def get_action(self, observation):
        #行動はbから取得
        #最後はpiを提出
        state = observation.board #埋まっている部分の可能性は他のところに山分けする

        state_key = self.generate_state_key(observation)
        #おけない部分の修正
        #山分けをあきらめない
        p = 0
        pool = 0
        tmp = {0:-1, 1:-1, 2:-1, 3:-1, 4:-1, 5:-1, 6:-1}
        for i in range(self.action_size):
           
            if state[i] != 0:
                #print("captured")
                tmp[i] = 0 
                pool += self.b[state_key][i]
            else:
                p += 1
        
        pool = pool / p
       
            
        for i in range(self.action_size):
            if state[i] == 0:
                tmp[i] = pool + self.b[state_key][i]
            else:
                tmp[i] = 0
        
        #if p != self.action_size:
            #print("challenge")
            #print("before", self.b[state_key])
            #print("after", tmp)
       
                
        self.b[state_key] = tmp
        action_probs = self.b[state_key]
        #print(action_probs)
       
        
        actions = list(action_probs.keys())
        probs = list(action_probs.values())
        
        return np.random.choice(actions, p=probs).tolist()
    
    def add(self, observation, action, reward):
        state_key = self.generate_state_key(observation)
        data = (state_key, action, reward)
        #print(data)
        self.memory.append(data)

    def reset(self):
        self.memory.clear()
    
    def greedy_probs(self, Q, state_key, epsilon):
        #print("greedy", state_key)
        p = 0
        tmp = []
        for action in range(self.action_size):
            if self.b[state_key][action] == 0:
                tmp.append(0)
            else:
                tmp.append(1)
                p += 1
        qs = [Q[(state_key, action)] for action in range(self.action_size)]
        max_action = np.argmax(qs)
        #おけない場所
        base_prob = epsilon / p
        #だめそうだったらprob作ってから
        action_probs = {action: 0.0 for action in range(self.action_size)}
        for action in range(self.action_size):
            if tmp[action] == 1:
                action_probs[action] += base_prob
        
        action_probs[max_action] += 1 - epsilon
        print("before greedy", self.pi[state_key])
        print("afrer greedy", action_probs)
        
        return action_probs
    
    def update(self):
        G = 0
        rho = 1
        for data in reversed(self.memory):
            state_key, action, reward = data
            rho = self.pi[state_key][action]/self.b[state_key][action]
            G = self.gamma * rho * G + reward
            key = (state_key, action)
            self.cnts[key] += 1 
            self.Q[key] += ( G - self.Q[key]) * self.alpha
            self.pi[state_key] = self.greedy_probs(self.Q, state_key, 0)
            self.b[state_key] = self.greedy_probs(self.Q, state_key, self.epsilon)
            
            
    
    
    

In [ ]:
env.reset()
# Play as the first agent against default "random" agent.
env.run([my_agent, "random"])
env.render(mode="ipython", width=500, height=450)

In [78]:
env = make("connectx", debug=True)
trainer = env.train(["random", None])
episodes = 1
agent = McOffPolicyAgent()
print(type(agent))


for episode in range(episodes):
    #print("new game")
    agent.reset()
    observation = trainer.reset()
    #print(type(observation.board))
    #env.render()
    
    
    while not env.done:
        print("key", agent.generate_state_key(observation))
        action = agent.get_action(observation)
        #print(type(action.tolist()))
        #past_observation = observation
        next_observation, reward, done, info = trainer.step(action)
        #print(type(reward), reward,done,info)
      
        agent.add( observation, action, reward)
        
        if done:
            #print(reward)
            agent.update()
            break
        
        observation = next_observation
        #print(observation.board)

print(agent.pi)
print(agent.b)
#print(agent.memory)
#print(agent.Q)

<class '__main__.McOffPolicyAgent'>
key 53
key 125c
key 12fe39
key 4559b21a
key 7388d4b5
key 13c48fbfe8900a
key 13c555063ec05f
key 13c555063f4084
key 20eff4cd0c94c7
key 20eff4cd24e8d0
key 11954c7286704c107
key 11954c72936daa9ea
before greedy {0: 0.14285714285714285, 1: 0.14285714285714285, 2: 0.14285714285714285, 3: 0.14285714285714285, 4: 0.14285714285714285, 5: 0.14285714285714285, 6: 0.14285714285714285}
afrer greedy {0: 1.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0}
before greedy {0: 1.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.0, 5: 0.0, 6: 0.0}
afrer greedy {0: 0.92, 1: 0.02, 2: 0.0, 3: 0.0, 4: 0.02, 5: 0.02, 6: 0.02}
before greedy {0: 0.14285714285714285, 1: 0.14285714285714285, 2: 0.14285714285714285, 3: 0.14285714285714285, 4: 0.14285714285714285, 5: 0.14285714285714285, 6: 0.14285714285714285}
afrer greedy {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 1.0, 5: 0.0, 6: 0.0}
before greedy {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 1.0, 5: 0.0, 6: 0.0}
afrer greedy {0: 0.02, 1: 0.02, 2: 0.0, 3: 0.0, 4

In [58]:
#print(agent.pi)

In [66]:
pi = agent.pi
p = {}

    
def my_agent(observation, configuration):
    import numpy as np
    from random import choice
    from collections import defaultdict
    pi = agent.pi # あとで変換
    state = observation.board[:]
    state.append(observation.mark)
    state_key = np.array(state).astype(str) 
    state_key = hex(int(''.join(state_key), 3))[2:]
    print(state_key)
    if state_key not in pi.keys():
        print("random")
        return choice([c for c in range(configuration.columns) if observation.board[c] == 0])
    
    actions = list(pi[state_key].keys())
    probs = list(pi[state_key].values())
    action = np.random.choice(actions, p=probs).tolist()
    
    return action

#print(dict(agent.pi))

In [68]:
import sys
out = sys.stdout

sys.stdout = out

env = make("connectx", debug=True)
env.run([my_agent, my_agent])
print("Success" if env.state[0].status == env.state[1].status == "DONE" else "Failed")

1
88d
random
8c2
8c6
random
e77
ec9
random
1dc1a
7b461de
random
838bedd39f
838bedd493
random
838bedd4a4
838bf5f096
random
838bf623d7
random
23248d7558e03
random
23248d7e784f4
random
23248d7e7d1d8
random
232496275398d
random
2324bd1f18c41
random
2324bd7133ac2
random
2324bd72b8e96
random
2324d76d3c5b7
random
2393cefdb4573
random
262d9c60843d4
random
26529990ac3be
random
2652999102bb3
random
1625193f6ae737
random
8ca4b7258b4c48
random
13e6423febbe3e4
random
13e64242ceb0675
random
5efbcbb079f0a2f9f
random
5efbcbb3375c80034
random
5efbcbb3376a2f2a0
random
5efbcbcbe034f47e5
random
7e9e6d9b47eaf0049
random
93b584258d0eed08a
random
94e1772d3b90ecde4
random
Success


In [ ]:
import sys
out = sys.stdout
sub = utils.read_file("/kaggle/working/submission.py")
print(callable("/kaggle/working/submission.py"))
agent = utils.call(submission)
print(agent)
print(type(agent))
sys.stdout = out

env = make("connectx", debug=True)
env.run([agent, agent])
print("Success" if env.state[0].status == env.state[1].status == "DONE" else "Failed")

In [61]:
my_agent = '''def my_agent(observation, configuration):
    import numpy as np
    from random import choice
    pi = '''\
    + str(dict(agent.pi)).replace(' ', '')\
    + '''
    state = observation.board
    state.append(observation.mark)
    state_key = np.array(state).astype(str) 
    state_key = hex(int(''.join(state_key), 3))[2:]
    if state_key not in pi.keys():
        return choice([c for c in range(configuration.columns) if observation.board[c] == 0])
    
    actions = list(pi[state_key].keys())
    probs = list(pi[state_key].values())
    action = np.random.choice(actions, p=probs).tolist()
    
    return action
    '''
with open('submission.py', 'w') as f:
    f.write(my_agent)

In [ ]:
env.reset()
# Play as the first agent against default "random" agent.
env.run([my_agent, "random"])
env.render(mode="ipython", width=500, height=450)

In [ ]:
import inspect
import os

def write_agent_to_file(function, file):
    with open(file, "a" if os.path.exists(file) else "w") as f:
        f.write(inspect.getsource(function))
        print(function, "written to", file)

write_agent_to_file(my_agent, "submission.py")